# MSc Simple Multi-Objective AutoML Pipeline

First I will look at 2 sets of data, 1 binary classification and one multiclass. I will do a complete pipeline and evaluation that will later be automated. That pipeline will include:

* Data Preprocessing
* Model Training
* Hyperparameter optimization.
* Model Evaluation with Accuracy
* Model Evaluation with Interpretability (specified metrics)
* Incorporating into a ranking function for model selection

Each automation will be constrained.

<img src="images\Pipeline.png"/>

Algorithms that will be used for modelling will be limited to:

1. Logistic Regression
2. Decision Trees
3. Random Forest
4. LightGBM
5. SVM

## Preprocessing Data

This study will assume that datasets input into the automated pipeline will be reasonably clean and representative, as commonly provided by machine learning dataset repositories. Therefore extensive data cleaning and feature engineering are considered outside the scope of this work.

In [1]:
# Importing the libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Import the algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import lightgbm as lgb

Telco Customer Churn:

https://www.kaggle.com/datasets/blastchar/telco-customer-churn 

In [2]:
# Import dataset
churn_data = pd.read_csv("Datasets/Telco-Customer-Churn.csv")
print(type(churn_data))

<class 'pandas.core.frame.DataFrame'>


In [3]:
# Check the shape of the dataframe
churn_data.shape

(7043, 21)

In [4]:
# Inspect the datatype of each feature and if there are any missing values
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
# Summary statistics
churn_data.describe(include='object')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,7590-VHVEG,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [6]:
churn_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Quick things to realise when inspecting the data:

* The ID column will be useless in modelling and will need to be dropped consistently accross datasets, it can be seen that the **number of unique values = number of rows** therefore some logic may be implemented to decide when to drop.

* The column "TotalCharges" has 6531 unique values, after inspection, these are numerical charge values stored as strings, in the dtype object. it would be good to process these in the pipeline.

To maintain a simple and generic preprocessing pipeline, features with type object can be treated as categorical variables. It can be implemented that features with more than 20 distinct values can be dropped, as this represents high-cardinality and can negatively impact interpretability and explanation stability. No attempt will be made to coerce these features to numeric values, as this would require dataset specific cleaning logic and would defeat the point of implementing this across datasets.

In [7]:
# creating a function for identifying data types of each column for preprocessing, splitting them into groups

def get_feature_groups(data):

    categorical_columns = data.select_dtypes(include=["object", "string", "category"]).columns #detect all categorical column labels
    numeric_columns = data.select_dtypes(include="number").columns #detect all numerical column labels 
    bool_columns = data.select_dtypes(include="bool").columns #detect possible boolean columns

    numeric_columns = numeric_columns.union(bool_columns).tolist() #merge boolean in with numeric as they are 0 and 1 representations

    high_cardinality = data[categorical_columns].nunique() > 20 #isolate high cardinality features

    #columns_to_drop = high_cardinality[high_cardinality].index.tolist() #create a column to drop list
    categorical_columns = high_cardinality[~high_cardinality].index.tolist() #dropping high cardinality features from categorical list
    
    return numeric_columns, categorical_columns, #columns_to_drop


By creating a list for each type of variable, this will lead into transforming the columns and preparing for a scikit learn pipeline.

In [8]:
# Split the data into training and test sets
y = churn_data["Churn"]
X = churn_data.drop("Churn", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape


((5634, 20), (5634,), (1409, 20), (1409,))

In [9]:
numerical_columns, categorical_columns = get_feature_groups(X_train)

In [10]:
print(numerical_columns)
print(categorical_columns)

['SeniorCitizen', 'tenure', 'MonthlyCharges']
['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [11]:
# Create a column transformer

preprocesser = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_columns),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns)
    ],
    remainder = "drop",
    n_jobs=-1 # enable use of all cpu threads locally
)

Column transformer will help to transform the numerical features and the categorical features.

In [12]:
# Create a dictionary of models to iterate over

models = {
    "logreg": LogisticRegression(),
    "dt": DecisionTreeClassifier(),
    "rf": RandomForestClassifier(),
    "svm": LinearSVC(),
   # "lgb": lgb(),
}

In [13]:
# Create an iterative pipeline

results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ("preprocess", preprocesser),
        ("model", model)
    ])

    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_test, y_test)

    results[name] = score

In [14]:
print(results)

{'logreg': 0.8211497515968772, 'dt': 0.7196593328601846, 'rf': 0.78708303761533, 'svm': 0.8183108587650816}


While the previous pipeline it clean, if cross validation is used feature selection based on X_train will cause data leakage for each fold as the validation set would have been used to make transformations on the training data. Hence we want to implement feature selection and preprocessing of data for each training fold.

In [15]:
# Split the data into training and test sets

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(churn_data["Churn"]) # encode category labels
X = churn_data.drop("Churn", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5634, 20), (5634,), (1409, 20), (1409,))

In [16]:
# Create a transformer class to group features in first step of a pipeline

from sklearn.base import BaseEstimator, TransformerMixin

class GroupFeatures(BaseEstimator, TransformerMixin):

    # Constructor method
    def __init__(self, max_unique=20):
        self.max_unique = max_unique

    #Fit method learns column groupings and detects high cardinality --> metadata for future steps
    def fit(self, X, y=None):
        categorical = X.select_dtypes(include=["object", "string", "category"])
        numeric = X.select_dtypes(include="number")
        boolean = X.select_dtypes(include="bool")

        high_cardinality = categorical.nunique() > self.max_unique #Detects high cardinality above maximum value specified

        self.numeric_columns_ = numeric.columns.union(boolean.columns).tolist() #Creates numerical column list
        self.categorical_columns_ = high_cardinality[~high_cardinality].index.tolist() #Creates categorical column list - high cardinality features

        self.keep_columns_ = self.numeric_columns_ + self.categorical_columns_

        return self

    # passes through the data untransformed 
    def transform(self, X):
        return X[self.keep_columns_]

In [17]:
# ColumnTransformer for feature preprocessing
from sklearn.compose import make_column_selector

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), make_column_selector(dtype_include="number")), 
        ("cat", OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=["object", "string", "category"])),
    ],
    remainder="drop"
)

In [18]:
# Implement Cross-Validation Evaluation

from sklearn.model_selection import StratifiedKFold, cross_validate

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

for name, model in models.items():

    # Create pipeline that include GroupFeatures in first step
    pipeline = Pipeline([
    ("features", GroupFeatures(max_unique=20)),
    ("preprocess", preprocessor),
    ("model", model)
    ])

    cv_results = cross_validate(
    pipeline,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    return_estimator=True
    )

    results[name] = cv_results

In [19]:
results['logreg']['test_accuracy']

array([0.78881988, 0.81011535, 0.78615794, 0.80834073, 0.79928952])

In [20]:
results['logreg']

{'fit_time': array([0.1442647 , 0.06859374, 0.06281042, 0.05928993, 0.04544878]),
 'score_time': array([0.03778338, 0.04402351, 0.04322529, 0.01792431, 0.03620172]),
 'estimator': [Pipeline(steps=[('features', GroupFeatures()),
                  ('preprocess',
                   ColumnTransformer(transformers=[('num', StandardScaler(),
                                                    <sklearn.compose._column_transformer.make_column_selector object at 0x00000251A3F00210>),
                                                   ('cat',
                                                    OneHotEncoder(handle_unknown='ignore'),
                                                    <sklearn.compose._column_transformer.make_column_selector object at 0x00000251A5CB7110>)])),
                  ('model', LogisticRegression())]),
  Pipeline(steps=[('features', GroupFeatures()),
                  ('preprocess',
                   ColumnTransformer(transformers=[('num', StandardScaler(),
            

`RandomSearchCV()` will be used for hyperparameter optimisation and a parameter space needs to be defined, for example:

Logistic Regression → C, penalty

Decision Tree → max_depth, min_samples_leaf

Random Forest → n_estimators, max_depth

XGBoost/LightGBM → max_depth, learning_rate, n_estimators

In [ ]:
# Define param distributions

param_grid = {
    "logreg" : {},
    "dt" : {},
    "rf" : {},
    "svm" : {},
    "lgb" : {}
}